# `521H0489 - Hồ Hữu An`

# `Text Classificaiton`

In [5]:
import pandas as pd
from underthesea import word_tokenize

In [6]:
filename = 'sentimentVN.csv'
df = pd.read_csv(filename)
print(df.shape)
df.head(5)

(31460, 3)


,content,label,start
0,Áo bao đẹp ạ!,POS,5
1,Tuyệt vời,POS,5
2,2day ao khong giong trong,NEG,1
3,"Mùi thơm,bôi lên da mềm da",POS,5
4,"Vải đẹp, dày dặn",POS,5


In [7]:
# !pip install underthesea

In [8]:
max = 10000
X = df['content'][:max].astype(str)
y = df['label'][:max]
for i in range(5):
    print(X[i],end='\t')
    print(y[i])

Áo bao đẹp ạ!	POS
Tuyệt vời	POS
2day ao khong giong trong	NEG
Mùi thơm,bôi lên da mềm da	POS
Vải đẹp, dày dặn	POS


In [9]:
Xnew = [word_tokenize(sent,format='text').lower() for sent in X]
for i in range(10):
    print(Xnew[i])

áo bao đẹp ạ !
tuyệt_vời
2 day ao khong_giong trong
mùi thơm , bôi lên da mềm da
vải đẹp , dày dặn
hàng rất đẹp , rất chi là ưng_ý
chất_lượng sản_phẩm tốt , date dài
ăn_nói và thái_độ phục_vụ tốt
đóng_gói sản_phẩm chắc_chắn
tất_sờn hết ca chưa dùng mà vay r


In [9]:
from underthesea import pos_tag
sent = 'chàng trai ấy thích cô gái không xinh trong lớp học'
ws = word_tokenize(sent,format='text')
pos_tag(ws)

[('chàng', 'N'),
 ('trai', 'N'),
 ('ấy', 'P'),
 ('thích', 'V'),
 ('cô', 'Nc'),
 ('gái', 'N'),
 ('không', 'R'),
 ('xinh', 'A'),
 ('trong', 'E'),
 ('lớp_học', 'M')]

In [15]:
# !pip install Keras-Preprocessing

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\ProgramData\anaconda3\python.exe -m pip install --upgrade pip


In [10]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(Xnew)
#print(t.word_index)
#print(t.word_counts)

In [18]:
X = t.texts_to_matrix(Xnew, mode='count')
print(X.shape)
print(X[:3])

(10000, 3579)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,
                                                    test_size=0.25)
print(X_train.shape)
print(y_train.shape)

(7500, 3579)
(7500,)


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import pickle

# `Training`

# `Naive Bayes`

In [26]:
nb_model = MultinomialNB()
nb_model.fit(X_train,y_train)
y_pred = nb_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         NEG       0.60      0.67      0.64       483
         NEU       0.38      0.38      0.38       415
         POS       0.87      0.84      0.85      1602

    accuracy                           0.73      2500
   macro avg       0.62      0.63      0.62      2500
weighted avg       0.74      0.73      0.73      2500



## `Logistic Regression`

In [29]:
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
print("Logistic Regression Results:")
print(classification_report(y_test, lr_pred))

Logistic Regression Results:
              precision    recall  f1-score   support

         NEG       0.67      0.66      0.66       483
         NEU       0.38      0.29      0.33       415
         POS       0.85      0.90      0.88      1602

    accuracy                           0.76      2500
   macro avg       0.63      0.62      0.62      2500
weighted avg       0.74      0.76      0.74      2500



## `Support Vector Machine`

In [30]:
svm_model = SVC(random_state=42)
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)
print("Support Vector Machine Results:")
print(classification_report(y_test, svm_pred))

Support Vector Machine Results:
              precision    recall  f1-score   support

         NEG       0.68      0.69      0.68       483
         NEU       0.43      0.26      0.32       415
         POS       0.84      0.93      0.88      1602

    accuracy                           0.77      2500
   macro avg       0.65      0.62      0.63      2500
weighted avg       0.74      0.77      0.75      2500



# `Save models`

In [32]:
models = {
    'naive_bayes': nb_model,
    'logistic_regression': lr_model,
    'svm': svm_model
}

for name, model in models.items():
    with open(f'{name}_model.pkl', 'wb') as file:
        pickle.dump(model, file)

In [24]:
question = input()
print(question)
X_input = t.texts_to_matrix([question],mode='count')
print(X_input)
y_output = model.predict(X_input)
print(y_output)

tôi thích nó
[[0. 0. 0. ... 0. 0. 0.]]
['NEU']


# `Test`

In [11]:
import pickle
def load_model(model_name):
    with open(f'{model_name}_model.pkl', 'rb') as file:
        return pickle.load(file)

def predict_sentiment(question, model_name):
    model = load_model(model_name)
    X_input = t.texts_to_matrix([question], mode='count')
    y_output = model.predict(X_input)
    return y_output[0]

while True:
    print("Select a model for prediction:")
    print("1. Naive Bayes")
    print("2. Logistic Regression")
    print("3. Support Vector Machine")
    print("4. Exit")
    
    choice = input("Enter your choice (1-4): ")
    
    if choice == '4':
        print("End.")
        break
    
    if choice not in ['1', '2', '3', '4']:
        print("Invalid choice. Please try again.")
        continue
    
    model_names = ['naive_bayes', 'logistic_regression', 'svm']
    selected_model = model_names[int(choice) - 1]
    
    question = input("Enter the text for sentiment analysis: ")
    result = predict_sentiment(question, selected_model)
    print(f"\nQuestion: {question}\n")
    print(f"Predicted sentiment: {result}\n")

Select a model for prediction:
1. Naive Bayes
2. Logistic Regression
3. Support Vector Machine
4. Exit

Question: thằng chó

Predicted sentiment: NEG

Select a model for prediction:
1. Naive Bayes
2. Logistic Regression
3. Support Vector Machine
4. Exit
End.
